In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\Saidabrorkhon\ML_Lectures\mobile_train.csv")

In [4]:
df_test = pd.read_csv(r"C:\Users\Saidabrorkhon\ML_Lectures\mobile_test.csv")

In [3]:
df.shape

(2000, 21)

In [5]:
df_test.shape

(1000, 21)

In [6]:
df.head(2)

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2


In [7]:
df_test.head(2)

,id,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,...,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,1,1043,1,1.8,1,14,0,5,0.1,193,...,16,226,1412,3476,12,7,2,0,1,0
1,2,841,1,0.5,1,4,1,61,0.8,191,...,12,746,857,3895,6,0,7,1,0,0


In [8]:
test_sample = df_test.sample(n=500, random_state=42)

In [9]:
df_test = df_test.drop(test_sample.index)

In [10]:
df = pd.concat([df, test_sample], ignore_index=True)

In [11]:
df.shape

(2500, 22)

In [12]:
df_test.shape

(500, 21)